In [79]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common import NoSuchElementException
from selenium.common import TimeoutException
import numpy as np
from selenium.webdriver import ActionChains


In [15]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get(url)

AcceptCookies = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Accept all')]")))
AcceptCookies.click()

In [16]:
ReviewsTab = wait.until(EC.element_to_be_clickable((By.XPATH, "//div[contains(@class, 'SVWlSe') and contains(@class, 't35a5d')]//span[text()='Reviews']")))
ReviewsTab.click()

In [ ]:
reviews = driver.find_elements(By.CSS_SELECTOR, 'css-selector-for-reviews')

for review in reviews:
    print(review.text)

In [8]:
url='https://www.google.com/search?rlz=1C1GCEA_enHR1085HR1085&q=Casablanca%20Reviews&rflfq=1&num=20&stick=H4sIAAAAAAAAAONgkxI2sTQzMjUzsDC2NDUzMTMzN7W03MDI-IpRyDmxODEpJzEvOVEhKLUsM7W8eBErFkEAULFNy0QAAAA&rldimm=4962560839564667599&tbm=lcl&hl=en&sa=X&ved=0CAYQ5foLahcKEwiAzYbJzIKEAxUAAAAAHQAAAAAQCg&biw=1536&bih=729&dpr=1.25#lkt=LocalPoiReviews&arid=ChZDSUhNMG9nS0VJQ0FnSURSNWJlRUhnEAE'

In [14]:
driver.quit()


In [83]:


arr = []

total_reviews_element=driver.find_element(By.XPATH, "//*[@id='akp_tsuid_13']/div/div[1]/div/g-sticky-content-container/div/block-component/div/div[1]/div/div/div/div[1]/div/div/div[6]/g-flippy-carousel/div/div/ol/li[3]/span/div/div/div/div[1]/div/div[3]/div/div/div[1]/div[2]/div/div/span/span")
total_reviews_text = total_reviews_element.text
total_reviews = int(total_reviews_text.split()[0])
while len(arr) < total_reviews:
    elements = driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")
    arr.extend(elements)

    # Move the focus to the last element
    ActionChains(driver).move_to_element(elements[-1]).perform()

    # Wait for new elements to load
    time.sleep(2)

    print(len(arr))
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#arr=driver.find_elements(By.XPATH, "//*[@data-hveid='2']/following-sibling::*//div[@class='OA1nbd']")



326
652
978


In [ ]:
for i in range(len(arr)):
    print(arr[i].text)

Hospitable, welcoming, good food, great value for money, a hidden gem in a corner of the Zagreb business center. The interior reminds of 1980s kitsch chic, but don't be put off by that, just enjoy!
Reasonable price.!
Nice ambiance

Ordered Prawns with Rice 🍚 😋
Not enough if you are hungry..!
Food: 5/5  |  Service: 5/5  |  Atmosphere: 5/5
Taste okay. Very clean restaurant
It was a perfectly well catered wedding, lots of space on the dance floor, unimposing yet efficient staff, good food
Moroccan food 🇲🇦🇲🇦🇲🇦🇲🇦 very tasty
Very nice menu selection at medium buget price. Soup and nice salad always included. Really fast service without big queues.
Such a wonderful place to have a wedding :) We ourselves had ours there in December 2017, it is modern, great staff, a lot of space and really good fresh and self-made food :)
Super …More
Great place for lunch in your lunch brake. Good choice of tasty food and pleasant waitresses.
Nice place, service and food. Considering dimensions it's good for e